In [ ]:
data_path = "/content/drive/MyDrive/instances_images"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ================================
# Standard library
# ================================
import os
import random

# ================================
# Array / image handling
# ================================
import numpy as np
from PIL import Image

# ================================
# PyTorch
# ================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, Subset

# ================================
# TorchVision (transforms)
# ================================
from torchvision import transforms

# ================================
# Scikit-learn
# ================================
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)

# ================================
# Visualization
# ================================
import matplotlib.pyplot as plt


In [ ]:
data_path = "/content/drive/MyDrive/instances_images"


In [ ]:
# ----- Semilla -----
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Dispositivo (CPU o GPU si hay)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Transformaciones: redimensionar + tensor + normalización simple
transform = transforms.Compose([
    transforms.Resize((32, 32)),   # importante para que coincida con la CNN
    transforms.ToTensor()
    # Podríamos agregar Normalize si queremos, pero lo dejamos simple por ahora
])





class GraphImagesDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        # Guardamos todas las rutas de imagen (png/jpg/jpeg)
        valid_exts = (".png", ".jpg", ".jpeg")
        self.image_paths = [
            os.path.join(root_dir, f)
            for f in os.listdir(root_dir)
            if f.lower().endswith(valid_exts)
        ]

        # Definimos las clases y sus índices
        self.class_names = ['barabasi', 'watts', 'erdos']
        self.class_to_idx = {name: idx for idx, name in enumerate(self.class_names)}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        # Sacar el nombre de archivo
        filename = os.path.basename(img_path)
        prefix = filename.split('_')[0].lower()  # primera palabra

        if prefix not in self.class_to_idx:
            raise ValueError(f"Clase desconocida en archivo {filename}: {prefix}")

        label = self.class_to_idx[prefix]

        if self.transform:
            img = self.transform(img)

        return img, label


dataset = GraphImagesDataset(data_path, transform=transform)
print("Total de imágenes:", len(dataset))

train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(
    dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

print("Train:", len(train_ds), "Val:", len(val_ds))



batch_size = 64

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False
)


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Entrada: 3 x 32 x 32
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

        # Después de 2 maxpool(2):
        # 32x32 -> 16x16 -> 8x8, con 64 canales
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 3)  # 3 clases

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)   # 3x32x32 -> 32x16x16

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)   # 32x16x16 -> 64x8x8

        x = torch.flatten(x, 1)  # a vector

        x = F.relu(self.fc1(x))
        x = self.fc2(x)          # logits (sin softmax)
        return x

model = Net().to(device)
print(model)



def train_loop(n_epochs, model, optimizer, loss_fn, train_loader, device):
    model.train()
    for epoch in range(1, n_epochs + 1):
        running_loss = 0.0
        total = 0

        for imgs, labels in train_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)

            # 1. forward
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            # 2. backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * labels.size(0)
            total += labels.size(0)

        epoch_loss = running_loss / total
        print(f"Época {epoch}/{n_epochs} - Loss train: {epoch_loss:.4f}")



loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


n_epochs = 100  # puedes cambiarlo
train_loop(n_epochs, model, optimizer, loss_fn, train_loader, device)



Total de imágenes: 1440
Train: 1152 Val: 288
Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
)
Época 1/100 - Loss train: 1.0989
Época 2/100 - Loss train: 1.0987
Época 3/100 - Loss train: 1.0986
Época 4/100 - Loss train: 1.0987
Época 5/100 - Loss train: 1.0983
Época 6/100 - Loss train: 1.0981
Época 7/100 - Loss train: 1.0979
Época 8/100 - Loss train: 1.0976
Época 9/100 - Loss train: 1.0972
Época 10/100 - Loss train: 1.0975
Época 11/100 - Loss train: 1.0967
Época 12/100 - Loss train: 1.0966
Época 13/100 - Loss train: 1.0961
Época 14/100 - Loss train: 1.0956
Época 15/100 - Loss train: 1.0952
Época 16/100 - Loss train: 1.0947
Época 17/100 - Loss train: 1.0932
Época 18/100 - Loss train: 1.0928
Época 19/100 - Loss train: 1.0908
Época 20/100 - Loss train: 1.0884
É

In [ ]:

def validate(model, train_loader, val_loader, device):
    model.eval()
    with torch.no_grad():
        for name, loader in [("train", train_loader), ("val", val_loader)]:
            all_labels = []
            all_preds = []

            for imgs, labels in loader:
                imgs = imgs.to(device)
                labels = labels.to(device)

                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)

                # Guardar resultados para métricas
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())

            # Calcular métricas
            acc  = accuracy_score(all_labels, all_preds)
            prec = precision_score(all_labels, all_preds, average='macro')
            rec  = recall_score(all_labels, all_preds, average='macro')
            f1   = f1_score(all_labels, all_preds, average='macro')

            print(f"{name.capitalize()} Accuracy : {acc:.4f}")
            print(f"{name.capitalize()} Precision: {prec:.4f}")
            print(f"{name.capitalize()} Recall   : {rec:.4f}")
            print(f"{name.capitalize()} F1-score : {f1:.4f}")
            print("-"*40)

# Ejecutar
validate(model, train_loader, val_loader, device)



Train Accuracy : 0.9983
Train Precision: 0.9983
Train Recall   : 0.9983
Train F1-score : 0.9983
----------------------------------------
Val Accuracy : 0.9965
Val Precision: 0.9966
Val Recall   : 0.9966
Val F1-score : 0.9966
----------------------------------------


In [ ]:

# Definir etiquetas de clase explícitas
class_labels = ['BA','ER','WS']

def evaluar_con_matriz(model, loader, device, nombre="Conjunto", modelo="cnn"):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    # Métricas
    acc  = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average='macro')
    rec  = recall_score(all_labels, all_preds, average='macro')
    f1   = f1_score(all_labels, all_preds, average='macro')

    print(f"{nombre} Accuracy : {acc:.4f}")
    print(f"{nombre} Precision: {prec:.4f}")
    print(f"{nombre} Recall   : {rec:.4f}")
    print(f"{nombre} F1-score : {f1:.4f}")
    print("-"*40)

    # Matriz de confusión con etiquetas de clase
    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=class_labels)
    disp.plot(cmap="Blues" if nombre=="Train" else "Oranges", values_format="d")
    plt.title(f"{nombre}")

    # Guardar gráfico con nombre del modelo y conjunto
    filename = f"{modelo}_{nombre.lower()}_confusion.png"
    plt.savefig(filename, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"Gráfico guardado en: {filename}")

# Ejecutar para train y val
evaluar_con_matriz(model, train_loader, device, nombre="Train", modelo="cnn")
evaluar_con_matriz(model, val_loader, device, nombre="Test", modelo="cnn")




Train Accuracy : 0.9983
Train Precision: 0.9983
Train Recall   : 0.9983
Train F1-score : 0.9983
----------------------------------------
Gráfico guardado en: cnn_train_confusion.png
Test Accuracy : 0.9965
Test Precision: 0.9966
Test Recall   : 0.9966
Test F1-score : 0.9966
----------------------------------------
Gráfico guardado en: cnn_test_confusion.png
